Today we're going to utilize a very simple (but rich) data set housed in the UCI Machine Learning repository. The Adult Income Dataset is taken from US Census information and is formatted particularly well to study the features/regressors/predictors that go into determining whether an adult US resident is 'likely' to have a household income greater than $50,000. 

The data includes age, workclass, a weight variable (to account for the unbalanced sampling), education level, time spent in education (in years), marital status, occupation, relationship, race, sex, individuals residency, and a target column that indicates whether the person attained a household income greater than $50,000. All in all, an interested data set for socio-economic research. So let's get our hands dirty and load up some data!

In [1]:
from sklearn import naive_bayes
import pandas as pd
import numpy as np
import matplotlib as plt

# Load the data 

Load the adult data set, which is just .txt file. There are no column labels. Read the docs for the data set here: https://archive.ics.uci.edu/ml/datasets/Adult, and use the in-built Pandas dataframe options to attach the column labels into the data frame. 

In [9]:
read_csv_data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', header=None)

In [10]:
read_csv_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [14]:
read_csv_data.columns = ['age','workclass','fnlwgt','education','education-num',
                         'marital-status','occupation','relationship', 'race', 'sex',
                         'capital-gain','capital-loss','hours-per-week','native-country', 'target']
read_csv_data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


# Convert the categorical variables into unordered integral values

For us to use the scikit-learn (although not every implementation of) Naive Bayes, we must pass in numerical data. Since we have decided to analyze all unordered categorical values, we can do a one-hot encoding to convert our categorical data into a numerical data frame.

**Note**: Do not use scikit-learn's implementation of One-hot encoding, we want to get you familiar with a bunch of methods, but as you should know by now, there are many ways to do the same thing. If you want, to a challenge, you can write the procedure both from scikit-learn and Pandas method. 

In [21]:
adult_dat = read_csv_data
Sex = pd.get_dummies(adult_dat['sex'])
Workclass = pd.get_dummies(adult_dat['workclass']) 
Education = pd.get_dummies(adult_dat['education'])
Marital = pd.get_dummies(adult_dat['marital-status'])
Occupation = pd.get_dummies(adult_dat['occupation'])
Relationship = pd.get_dummies(adult_dat['relationship'])
Race = pd.get_dummies(adult_dat['race'])
Country = pd.get_dummies(adult_dat['native-country'])
Target = pd.get_dummies(adult_dat['target'])

In [27]:
# Clean up the data set by deleting un-used columns

one_hot_dat = pd.concat([adult_dat, Sex, Workclass, Marital, Occupation, Relationship, Race, Country, Target], axis = 1)
del one_hot_dat['sex']
del one_hot_dat['workclass']
del one_hot_dat['education']
del one_hot_dat['marital-status']
del one_hot_dat['occupation']
del one_hot_dat['relationship']
del one_hot_dat['race']
del one_hot_dat['native-country']
del one_hot_dat['target']
del one_hot_dat['>50K']
one_hot_dat.head()

KeyError: '>50K'

# Challenge Problem: Alternative Encoding Scheme to One-Hot Encoding

Likewise, beside doing a One-hot encoding, we could also map each string label in our categorical features to a integral value. As we previously leveraged a Pandas data frame method to do the encoding, we are now going to test out a scikit-learn method to impose the integral value encoding. Please check the docs and read up on: http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html. Proceed with the encoding and build a Naive Bayes and Logistic classifier for both. Do we get similar results? What should we expect? And why?

In [29]:
import seaborn as sns
from sklearn import preprocessing

the_labelizer = preprocessing.LabelEncoder()
adult_dat_2 = adult_dat

def integral_encoding(table):
    the_labelizer = {}
    
    for col in table.columns:
        if table.dtypes[col] == np.object:
            the_labelizer[col] = preprocessing.LabelEncoder()
            table[col] = the_labelizer[col].fit_transform(table[col])
            
    return table

integral_df = integral_encoding(adult_dat_2)

# Summarize the data and engage in elementary data exploration

For some data exploration, use Pandas histogram methods to display the features. 

In [125]:
# Write histogram functions here, and/or any other data visualizations

# Partition the data

Without using any direct method/libraries that would automatically accomplish this, please partition the data set 70/30. You can use anything from the math, pandas, or numpy library, do not use other libraries. 

In [30]:
partition_val = np.random.rand(len(integral_df)) < 0.70
train = integral_df[partition_val]
test = integral_df[~partition_val]
print train.shape
print test.shape

(22868, 15)
(9693, 15)


In [31]:
train.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,39,7,77516,9,13,4,1,1,4,1,2174,0,40,39,0
1,50,6,83311,9,13,2,4,0,4,1,0,0,13,39,0
6,49,4,160187,6,5,3,8,1,2,0,0,0,16,23,0
9,42,4,159449,9,13,2,4,0,4,1,5178,0,40,39,1
10,37,4,280464,15,10,2,4,0,2,1,0,0,80,39,1


# Define your feature set and define your target 

In [32]:
target_train = train['target']
feature_train = train.drop('target', axis=1)


# Run Naive Bayes Classifier

Instantiate the Naive Bayes predictor from scikit-learn with the training data. 

In [33]:
Cat_Naive_Bayes = naive_bayes.MultinomialNB();
Cat_Naive_Bayes.fit(feature_train, target_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

# Check Accuracy / Score for Naive Bayes

Define the target and feature set for the test data

In [34]:
target_test = test['target']
feature_test =  test.drop('target', axis = 1)

Score the Naive Bayes classifier on the test data

In [35]:
Cat_Naive_Bayes.score(feature_test, target_test)

0.77849994841638293

# Check Accuracy / Score for a Logistic Classifier 

Define a logistic regression and train it with the feature and target set

In [37]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(feature_train, target_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

Produce the accuracy score of the logistic regression from the test set

In [38]:
logreg.score(feature_test, target_test)

0.79088001650675743

Was that what you expected? All we did was remove non categorical variables, and imposed a One-hot encoding, should we have expected the Naive Bayes to underperform the Logistic? Here are some other things you can think about:

1. What other metrics outside of simple accuracy can we utilize to measure performance?
2. Could some pair-wise correlation between pair-wise features in our feature set have caused an issue with the Naive Bayes? What are the assumptions for Naive Bayes which may cause this to happen? 
3. How could we improve the performance of Naive Bayes? 
4. What about the numerica features we left out, should we bring them back in? How?

If you want to expand on your analysis, why not build a correlation matrix, or perhaps print a summary of the logistic regression, would an ANOVA table help in our assessment for this case? 